# Assignment 9: Multivariate and Machine Learning Analysis for Intracranial EEG Data
Please submit this assignment to Canvas as a jupyter notebook (.ipynb).  The assignment will have you utilize machine learning techniques to classify memory states.

In [1]:
# imports
import numpy as np
import pandas as pd
import cmlreaders as cml
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold
from scipy.stats import ttest_ind

## Question 1, Optimizing hyperparameters using nested cross-validation

In the previous assignment you used a fixed penalty parameter (C = 1) for all subjects. Oftentimes we want to tune the hyperparameters of a model to optimize its performance. It is crucial to make it clear that the aim of cross-validation is not to get one or multiple trained models for inference, but to estimate an unbiased generalization performance. We can use a grid search approach in which we search over a grid of 10 values of C logarithmically spaced between $10^{−6}$ and $10^2$ (np.logspace(-6,2,10)). One naive approach is to repeat Part I for every C value and select the optimal C that maximizes the average AUC across folds. The problem is, if we use the test set multiple times for different trained models, during our selection of the optimal model, the test set actually “leaks” information, and is thus impure or biased. To rigorously select the optimal parameter C and correctly estimate the prediction error of the optimal model, we utilize a nested cross-validation procedure. As the name suggests, you will perform two rounds of cross-validation with the inner CV nested in the outer CV. The outer CV is responsible for obtaining the prediction error for the model and the inner CV is responsible for selecting the optimal hyperparameter for each outer CV fold. Apply the nested cross-validation procedure from the notes to the dataset.

Does optimizing the regularization hyperparameter help? Use barplots, scatterplots, and appropriate statistical tests to support your conclusions.

In [2]:
# Question 1
### YOUR CODE HERE

## Question 2, Comparison between different penalization schemes and z-scoring features

1. In this section you will investigate how different penalization schemes and how z-scoring features can produce different behaviors in the classifier. Recall that the objective function for penalized logistic regression is: 

$l(\beta) = \Sigma_{i=1}^N{y_i log p_i + (1 − y_i) log(1 − p_i)} + \frac{\alpha}{2} r||\beta||_2^2 + \alpha(1 − r)||\beta||_1$

where, $\alpha = 1/C$ is the penalty parameter, r is the contribution of L2 penalty, and 1 − r is the contribution of L1 penalty. When r = 1, we have a strictly L2 penalized logistic regression. When r = 0, we have a strictly L1 penalized regression (a.k.a. Lasso). When 0 < r < 1, we have a mixture of both L1 and L2, which is called elastic net. In this part, you will compare the performances of different penalization schemes: strictly L2, strictly L1, and elastic net with r = 0.5. Repeat the nested cross-validation procedure in Part 2
using z-scored features for L2, L1, and elastic net. You should again use sklearn’s linear_model.LogisticRegression class for your classifier, appropriately selecting the classifier hyperparameters to obtain the L1 and elastic net regularization schemes.
* Compare the performances (AUCs) across these three schemes using a barplot or whatever you see fit, including some visualization of variability in the outcomes for these methods. Does one scheme do better than the others?
* Keep in mind that the sklearn LogisticRegression parameter `C` is an inverse regularization strength, meaning the regularization strength is equal to $\alpha$ = 1/C. So higher `C` means lower regularization.

In [3]:
# Question 2
### YOUR CODE HERE

## Question 3

Use the *model.coef_* attribute (*model* is your classifier object) to investigate the learned coefficients of the classifier for each subject. You can pool classifier weights across all outer cross-validation folds. Generate a plot for each of the first three subjects containing three histograms (one for each penalization scheme) of the model coefficients (use the *alpha* parameter of the plt.hist function to ensure the histograms do not cover each other up). 

In [4]:
# Question 3
### YOUR CODE HERE

## Question 4, Does z-scoring improve performance?
* In this part, repeat the analysis from Part 2, Question 1 but without z-scored features. Which is better, raw features or z-scored features? Give an intuitive explanation as to why one is better than the other.

In [5]:
# Question 4
### YOUR CODE HERE